If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it. \\

Then you need to install Git-LFS. Uncomment the following instructions.

In [ ]:
%%capture
!pip install datasets transformers
!apt install git-lfs

In [ ]:
import huggingface_hub
from huggingface_hub import notebook_login
your_token = "ADD YOUR TOKEN HERE"
huggingface_hub.login(token = your_token)

# Fine-tuning a model on a text classification task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a text classification task of the [GLUE Benchmark](https://gluebenchmark.com/). We will focus on CoLA dataset.


The GLUE Benchmark is a group of nine classification tasks on sentences or pairs of sentences which are:

- [CoLA](https://nyu-mll.github.io/CoLA/) (Corpus of Linguistic Acceptability) Determine if a sentence is grammatically correct or not.is a  dataset containing sentences labeled grammatically correct or not.
- [MNLI](https://arxiv.org/abs/1704.05426) (Multi-Genre Natural Language Inference) Determine if a sentence entails, contradicts or is unrelated to a given hypothesis. (This dataset has two versions, one with the validation and test set coming from the same distribution, another called mismatched where the validation and test use out-of-domain data.)
- [MRPC](https://www.microsoft.com/en-us/download/details.aspx?id=52398) (Microsoft Research Paraphrase Corpus) Determine if two sentences are paraphrases from one another or not.
- [QNLI](https://rajpurkar.github.io/SQuAD-explorer/) (Question-answering Natural Language Inference) Determine if the answer to a question is in the second sentence or not. (This dataset is built from the SQuAD dataset.)
- [QQP](https://data.quora.com/First-Quora-Dataset-Release-Question-Pairs) (Quora Question Pairs2) Determine if two questions are semantically equivalent or not.
- [RTE](https://aclweb.org/aclwiki/Recognizing_Textual_Entailment) (Recognizing Textual Entailment) Determine if a sentence entails a given hypothesis or not.
- [SST-2](https://nlp.stanford.edu/sentiment/index.html) (Stanford Sentiment Treebank) Determine if the sentence has a positive or negative sentiment.
- [STS-B](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark) (Semantic Textual Similarity Benchmark) Determine the similarity of two sentences with a score from 1 to 5.
- [WNLI](https://cs.nyu.edu/faculty/davise/papers/WinogradSchemas/WS.html) (Winograd Natural Language Inference) Determine if a sentence with an anonymous pronoun and a sentence with this pronoun replaced are entailed or not. (This dataset is built from the Winograd Schema Challenge dataset.)

We will see how to easily load the dataset for each one of those tasks and use the `Trainer` API to fine-tune a model on it. Each task is named by its acronym, with `mnli-mm` standing for the mismatched version of MNLI (so same training set as `mnli` but different validation and test sets):

In [ ]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

This notebook is built to run on any of the tasks in the list above, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a classification head. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [ ]:
task = "rte"
model_checkpoint = "bert-base-uncased"
batch_size = 128

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [ ]:
#from datasets import load_dataset, load_metric

from datasets import load_dataset
!pip install evaluate
from evaluate import load

Apart from `mnli-mm` being a special code, we can directly pass our task name to those functions. `load_dataset` will cache the dataset to avoid downloading it again the next time you run this cell.

In [ ]:
actual_task = "mnli" if task == "mnli-mm" else task
print(f"Task is: {actual_task} ")
dataset = load_dataset("glue", actual_task)

!pip install scikit-learn
metric = load("accuracy")

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

In [ ]:
dataset

To access an actual element, you need to select a split first, then give an index:

In [ ]:
dataset["train"][0]

In [ ]:
dataset["test"][1]
#All the test labels are DUMMY

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"])

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

You can call its `compute` method with your predictions and labels directly and it will return a dictionary with the metric(s) value:

In [ ]:
import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("Hello, this one sentence!", max_length=50, padding="max_length", truncation=True)

In [ ]:
tokenizer("Hello, this one sentence!")

In [ ]:
tokenizer("Hello, this one sentence!","What is going on here!")

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

In [ ]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

We can double check it does work on our current dataset. Recall the dataset object again:

In [ ]:
dataset['test'][1]

In [ ]:
print(dataset['test']['label'])

In [ ]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model.

In [ ]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
preprocess_function(dataset['train'][:1])


To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about sentence classification, we use the `AutoModelForSequenceClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which is always 2, except for STS-B which is a regression problem and MNLI where we have 3 labels):

In [ ]:
encoded_dataset

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from transformers import logging

logging.set_verbosity_error() #Since we learn the meaning of warning, we can silence it.

num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2

model_checkpoint = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
model.dropout.p = 0.4


In [ ]:
print(model)

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define two more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]
run_no="3"
args = TrainingArguments(
    f"{model_name}-finetuned-{task}-run_{run_no}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the `Trainer` to load the best model it saved (according to `metric_name`) at the end of training.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name. Otherwise, it will be pushed directly to your own account.

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B):

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

You might wonder why we pass along the `tokenizer` when we already preprocessed our data. This is because we will use it once last time to make all the samples we gather the same length by applying padding, which requires knowing the model's preferences regarding padding (to the left or right? with which token?). The `tokenizer` has a pad method that will do all of this right for us, and the `Trainer` will use it. You can customize this part by defining and passing your own `data_collator` which will receive the samples like the dictionaries seen above and will need to return a dictionary of tensors.

We can now finetune our model by just calling the `train` method:

In [ ]:
# Recall that we have CUDA device. Lets check the amount of memory.
!nvidia-smi

In [ ]:
trainer.train()

We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one):

In [ ]:
trainer.evaluate()

To see how your model fared you can compare it to the [GLUE Benchmark leaderboard](https://gluebenchmark.com/leaderboard).

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
%%capture
trainer.push_to_hub()  #THIS COMMAND PUSHES THE HUGGING FACE THE LAST MODEL

Now you can always download your model with `AutoModelForSequenceClassification` with the model name.

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("kocemirhan/bert-base-uncased-finetuned-rte-run_3")
#check your own model also
print(model)

## Hyperparameter search

The `Trainer` supports hyperparameter search using [optuna](https://optuna.org/) or [Ray Tune](https://docs.ray.io/en/latest/tune/). For this last section you will need either of those libraries installed, just uncomment the line you want on the next cell and run it. I prefer Optuna.

In [ ]:
%%capture
! pip install optuna
# ! pip install ray[tune]

During hyperparameter search, the `Trainer` will run several trainings, so it needs to have the model defined via a function (so it can be reinitialized at each new run) instead of just having it passed. We just use the same function as before:

In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)



In [ ]:
import optuna
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# ----- 1. Custom Tokenization Function -----
def custom_tokenize(sample, seq_limit):
    # we need to tokenize custom due to the fact t
    # Tokenize based on whether a second sentence exists.
    if sentence2_key is None:
        return tokenizer(sample[sentence1_key],
                         max_length=seq_limit,
                         padding="max_length",
                         truncation=True)
    return tokenizer(sample[sentence1_key],
                     sample[sentence2_key],
                     max_length=seq_limit,
                     padding="max_length",
                     truncation=True)

# ----- 2. Model Initialization with Dynamic Dropout Value  -----
def init_model(trial: optuna.trial.Trial):
    # Load the model from the pretrained checkpoint with the appropriate number of labels.
    model_instance = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
    # Set dropout dynamically using the trial suggestion; default to 0.1 if no trial provided.
    dropout_val = trial.suggest_float("dropout", 0.0, 0.2, log=False) if trial else 0.1
    model_instance.dropout.p = dropout_val
    return model_instance

# ----- 3. Trainer Builder with Dynamic Tokenization -----
def build_trainer(trial):
    bs = 128  # Batch size remains fixed across all trials since it has not been considered in the assignment instructions.

    # Sample hyperparameters from the trial.
    seq_limit = trial.suggest_categorical("max_length", [64, 128, 256])
    num_epochs = trial.suggest_categorical("num_train_epochs", [3, 5, 10, 15])
    lr = trial.suggest_float("learning_rate", 1e-5, 1e-4, log=True)

    # Re-tokenize the dataset using the sampled maximum sequence length.
    tokenized_dataset = dataset.map(lambda x: custom_tokenize(x, seq_limit), batched=True)

    training_args = TrainingArguments(
        output_dir=f"{model_checkpoint.split('/')[-1]}-finetuned-{task}-run_{1}",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=lr,
        per_device_train_batch_size=bs,
        per_device_eval_batch_size=bs,
        num_train_epochs=num_epochs,
        weight_decay=0.0,
        load_best_model_at_end=True,
        metric_for_best_model=metric_name,
        push_to_hub=True,
    )

    trainer_instance = Trainer(
        model_init=lambda: init_model(trial),
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset[validation_key],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    return trainer_instance

# ----- 4. Optuna Objective Function -----
def objective_function(trial):
    trainer_obj = build_trainer(trial)
    trainer_obj.train()
    results = trainer_obj.evaluate()
    # Return evaluation accuracy as the metric to maximize.
    return results["eval_accuracy"]

# ----- 5. Run Hyperparameter Search -----
hyper_study = optuna.create_study(direction="maximize")
hyper_study.optimize(objective_function, n_trials=48)

print("Best trial found:")
best_trial = hyper_study.best_trial
print(f"  Eval Accuracy: {best_trial.value}")
print("  Best Hyperparameters:")
for param, val in best_trial.params.items():
    print(f"    {param}: {val}")




And we can instantiate our `Trainer` like before:

I want to list some additional resources that you might want to check.


*   [Simple Training with the Huggingface Transformers Trainer](https://www.youtube.com/watch?v=u--UVvH-LIQ&t=174s&ab_channel=HuggingFace)
*   



In [ ]:
# ----- 6. Final Model Training with Best Hyperparameters -----
final_trainer = build_trainer(best_trial)
final_train_output = final_trainer.train()  # Capture training output for loss logging.
final_eval = final_trainer.evaluate()
print("Final evaluation metrics:", final_eval)

In [ ]:
# ----- 7. Report Loss Curves and Accuracy Results of the Best-Performing Model -----
import matplotlib.pyplot as plt

# Extract the logged training history from the Trainer's state
log_history = final_trainer.state.log_history

# Initialize lists to store loss information.
train_steps, train_losses = [], []
eval_steps, eval_losses = [], []

# Process the logs.
for log in log_history:
    # Sometimes logs contain a "step" and "loss" entry during training.
    if "loss" in log and "step" in log:
        train_steps.append(log["step"])
        train_losses.append(log["loss"])
    # Evaluation logs usually have "eval_loss" and "step" entries.
    if "eval_loss" in log and "step" in log:
        eval_steps.append(log["step"])
        eval_losses.append(log["eval_loss"])

# Plotting the training and evaluation loss curves (if evaluation loss exists)
plt.figure(figsize=(10, 6))
plt.plot(train_steps, train_losses, label="Training Loss", marker="o", linestyle="-")
if eval_losses:
    plt.plot(eval_steps, eval_losses, label="Evaluation Loss", marker="o", linestyle="-")

plt.title("Loss Curves During Training")
plt.xlabel("Training Step")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

# Display the final evaluation metrics, including accuracy.
print("Final Evaluation Metrics:")
for metric, value in final_eval.items():
    print(f"{metric}: {value}")

# Optionally, specifically print the final evaluation accuracy if it exists.
if "eval_accuracy" in final_eval:
    print(f"\nFinal Evaluation Accuracy: {final_eval['eval_accuracy']}")

# Display the best trial hyperparameters
print("\nBest Trial Hyperparameters:")
for param, val in best_trial.params.items():
    print(f"  {param}: {val}")


In [ ]:
print("Best trial found:")
best_trial = hyper_study.best_trial
print(f"  Eval Accuracy: {best_trial.value}")
print("  Best Hyperparameters:")
for param, val in best_trial.params.items():
    print(f"    {param}: {val}")


In [ ]:
import matplotlib.pyplot as plt

# --- Plot Hyperparameter Tuning Results ---
trial_numbers = [trial.number for trial in hyper_study.trials]
eval_accuracies = [trial.value for trial in hyper_study.trials]

plt.figure(figsize=(10, 5))
plt.plot(trial_numbers, eval_accuracies, marker='o', linestyle='-', color='blue')
plt.xlabel("Trial Number")
plt.ylabel("Evaluation Accuracy")
plt.title("Hyperparameter Tuning Results")
plt.grid(True)
plt.show()



In [ ]:
final_trainer.push_to_hub(

    commit_message="Add final model from best trial found",

    blocking=True,

)

In [ ]:
import optuna.visualization as vis

In [ ]:
# Plot the parameter importances to see which hyperparameters had the highest impact on performance
fig_importance = vis.plot_param_importances(hyper_study)
fig_importance.update_layout(title="Optuna Parameters")
fig_importance.show()


In [ ]:
# Plot the optimization history (objective value vs. trial number)
fig_history = vis.plot_optimization_history(hyper_study)
fig_history.update_layout(title="Optuna History")
fig_history.show()


In [ ]:
# Generate a slice plot to visualize the relationship between hyperparameter values and evaluation metric
fig_slice = vis.plot_slice(hyper_study)
fig_slice.update_layout(title="Optuna Slice Plot")
fig_slice.show()


In [ ]:

# Create a parallel coordinate plot to inspect the multidimensional search space
fig_parallel = vis.plot_parallel_coordinate(hyper_study)
fig_parallel.update_layout(title="Optuna Parallel Coordinate Plot")
fig_parallel.show()